если шо, то этот код тоже пойдет в проект, но как гавнокод

In [1]:
all_tags_with_id = []
cols = steamspy.columns
for i in range(len(steamspy)):
    row = steamspy.loc[i]
    tags_with_id = {}
    tags = {}
    for elem in row:
        # так как DataFrame похож на One-Hot Encoding, можно брать только те значения,
        # которые не равны 0.
        if elem != 0:
            index = pd.Index(row).get_loc(elem)
            col = cols[index]
            # из-за того, что появляются списки с несколькими названиями столбцов
            # пришлось отбросить все, кроме первого, так как первым может быть 
            # id приложения.
            if 'pandas.core.indexes.base.Index' in str(type(col)):
                col = str(col[0])
            if col == 'appid':
                tags_with_id[col] = elem
            if col != 'appid':
                tags[col] = elem
    tags_with_id['tags'] = tags
    all_tags_with_id.append(tags_with_id)

NameError: name 'steamspy' is not defined

In [ ]:
def replace_nums(str):
    num = ''
    new_str = 
    for i in range(len(str)-1):
        if str[i] in '0123456789':
            num += str[i]
            # будем надеяться, что в наборе не будет чисел в конце текста
            # иначе выбросится ошибка
            if str[i+1] not in '0123456789':
                new_str += str.replace(num, 'NUM')
                num = ''
    return new_str

In [ ]:
def counter_to_feature(dataset, vocab_size):
    # все ради сокращения кода и функции most_common
    total_counts = Counter()
    vocabulary = {}
    for data in dataset:
        for word, count in data.items():
            # кажется, что бессмысленно
            # но на самом деле мы в одном словаре собираем все словари
            total_counts[word] += count
    # так как иначе можно получить бесконечную матрицу признаков.
    # а так у нас установленное значение
    most_common = total_counts.most_common()[:vocab_size]
    for index, word in enumerate(most_common):
        vocabulary[word] = index 
    return total_counts

Если чо, то некоторый код был взят отсюда: https://imdb-api.com/API#IMDbList-header (в сам)

In [ ]:
def get_similars(item_id):
    # для того, чтобы затем можно было вставить ID в DataFrame
    url = 'https://imdb-api.com/en/API/Title/k_65c2g6hh/' + item_id
    response = requests.request('GET', url)
    # код сворован, признаюсь.
    cleared_dict = json.loads(response.text.encode('utf-8').decode('utf-8'))
    # сразу получаем похожих, так как нам только они нужны
    # что будет, если их меньше чем 7? - ошибка
    recs = cleared_dict['similars'][:7]
    return recs

In [ ]:
def get_item_all_fields(endpoint, item_id):
    byte_array = wrapper.api_request(endpoint, 'fields *; where id = ' + str(item_id) + ';')
    decoded_bytes_array = byte_array.decode('utf-8')
    item = json.loads(decoded_bytes_array)
    if len(item) == 0:
        return 'This ID don`t exists.'
    if len(item) > 0:
        return item[0]

In [ ]:
for item_id in range(1, 160000):
    item = get_item_all_fields('games', item_id)
    if item == 'This ID don`t exists' or item['name'] in games_dataset.name.unique():
        continue
    if item['name'] not in games_dataset.name.unique():
        games_dataset.loc[len(games_dataset.index)] = item
    if item_id > 10:
        break

In [ ]:
def API_threading(floors, ceilings):
    process = []
    with ProcessPoolExecutor(max_workers=8) as executor:
        [process.append(executor.submit(create_games_dataset, floor, ceiling)) for floor in floors for ceiling in ceilings]
    for task in as_completed(process):
        return task.result()